<a href="https://colab.research.google.com/github/opheliaophie/Hacking-Books/blob/master/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RPC Endpoint: https://api.mainnet-beta.solana.com

### (deprecated) Monitor Wallet Activity and detect token creation and liquidity

In [ ]:
import asyncio
import json
import requests
import websockets
import nest_asyncio

# Apply nest_asyncio to avoid loop issues in Google Colab
nest_asyncio.apply()

# Public Solana RPC endpoint
RPC_URL = "https://api.mainnet-beta.solana.com"
WS_URL = "wss://api.mainnet-beta.solana.com"

# Target wallet to monitor
TARGET_WALLET = "HYoXu9V94BxGCWmiE47R5B6Zcb6seXPjQjhkKcKP5gBW"

# Define thresholds
TRACKED_AMOUNTS = {-4: 3.7, -1: 0.7}  # Starting and destination amounts
LIQUIDITY_THRESHOLD = -300  # Large liquidity deposits

# Store wallet tracking information
tracked_wallets = {}

async def subscribe_to_account(wallet_address):
    """Connects to Solana WebSocket and listens for transactions."""
    async with websockets.connect(WS_URL) as ws:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "accountSubscribe",
            "params": [wallet_address, {"encoding": "jsonParsed"}]
        }
        await ws.send(json.dumps(payload))

        print(f"Listening for transactions from {wallet_address}...")

        while True:
            try:
                response = await ws.recv()
                data = json.loads(response)

                if "result" in data and "value" in data["result"]:
                    await fetch_transaction_details(wallet_address)
            except Exception as e:
                print(f"[ERROR] WebSocket error: {e}")
                break  # Restart connection if needed

async def fetch_transaction_details(wallet_address):
    """Fetches the latest transactions for the target wallet asynchronously."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getSignaturesForAddress",
            "params": [wallet_address, {"limit": 5}]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response:
            for tx in response["result"]:
                await fetch_transaction(tx["signature"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transactions: {e}")

async def fetch_transaction(signature):
    """Fetches transaction details asynchronously and extracts relevant transfers."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTransaction",
            "params": [signature, "jsonParsed"]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response and "meta" in response["result"]:
            process_transaction(response["result"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transaction details: {e}")

def process_transaction(transaction):
    """Analyzes the transaction to detect rug pull patterns and track wallets."""
    try:
        if "meta" in transaction and "postTokenBalances" in transaction["meta"]:
            instructions = transaction["transaction"]["message"]["instructions"]
            for instruction in instructions:
                if "parsed" in instruction and "info" in instruction["parsed"]:
                    info = instruction["parsed"]["info"]
                    sender = info.get("source")
                    receiver = info.get("destination")
                    amount = float(info.get("lamports", 0)) / 1e9  # Convert from lamports to SOL

                    if sender and receiver:
                        print(f"🔍 Transfer detected: {amount} SOL from {sender} to {receiver}")

                        if sender in tracked_wallets:
                            tracked_wallets[receiver] = tracked_wallets[sender] - 0.1  # Decrease by 0.1 SOL
                            print(f"📌 Tracking {receiver}, expected balance: {tracked_wallets[receiver]} SOL")

                            if tracked_wallets[receiver] == 3.7:
                                print(f"🚀 Token Creator Wallet Identified: {receiver}")
                                track_token_creation(receiver)
                            elif tracked_wallets[receiver] == 0.7:
                                print(f"💀 Rug Pull Wallet Identified: {receiver}")
                                track_rug_pull(receiver)

                        if amount in TRACKED_AMOUNTS:
                            tracked_wallets[receiver] = amount + 0.1  # Initialize tracking
                            print(f"👀 Tracking new suspicious transfer chain: {receiver}")
    except Exception as e:
        print(f"[ERROR] Failed to process transaction: {e}")

def track_token_creation(wallet):
    """Monitors the wallet to detect when it creates a new SPL token."""
    print(f"🔎 Monitoring {wallet} for SPL token creation...")
    # TODO: Implement SPL token tracking logic

def track_rug_pull(wallet):
    """Monitors the rug pull wallet for large transfers."""
    print(f"🚨 Monitoring {wallet} for large fund movements...")
    # TODO: Implement tracking logic for fund extractions

async def main():
    """Runs the WebSocket listener asynchronously."""
    await subscribe_to_account(TARGET_WALLET)

# Run the event loop in Colab safely
loop = asyncio.get_event_loop()
if loop.is_running():
    print("Event loop already running. Running in background...")
    task = loop.create_task(main())
else:
    loop.run_until_complete(main())


Event loop already running. Running in background...


In [ ]:
import asyncio
import json
import requests
import websockets
import nest_asyncio

# Apply nest_asyncio to avoid loop issues in Google Colab
nest_asyncio.apply()

# Public Solana RPC endpoint
RPC_URL = "https://api.mainnet-beta.solana.com"
WS_URL = "wss://api.mainnet-beta.solana.com"

# Target wallet to monitor
TARGET_WALLET = "HYoXu9V94BxGCWmiE47R5B6Zcb6seXPjQjhkKcKP5gBW"

# Define thresholds
TRACKED_AMOUNTS = {-4: 3.7, -1: 0.7}  # Starting and destination amounts
LIQUIDITY_THRESHOLD = -300  # Large liquidity deposits

# Store wallet tracking information
tracked_wallets = {}

async def subscribe_to_account(wallet_address):
    """Connects to Solana WebSocket and listens for transactions."""
    async with websockets.connect(WS_URL) as ws:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "accountSubscribe",
            "params": [wallet_address, {"encoding": "jsonParsed"}]
        }
        await ws.send(json.dumps(payload))

        print(f"Listening for transactions from {wallet_address}...")

        while True:
            try:
                response = await ws.recv()
                data = json.loads(response)

                if "result" in data and "value" in data["result"]:
                    await fetch_transaction_details(wallet_address)
            except Exception as e:
                print(f"[ERROR] WebSocket error: {e}")
                break  # Restart connection if needed

async def fetch_transaction_details(wallet_address):
    """Fetches the latest transactions for the target wallet asynchronously."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getSignaturesForAddress",
            "params": [wallet_address, {"limit": 5}]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response:
            for tx in response["result"]:
                await fetch_transaction(tx["signature"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transactions: {e}")

async def fetch_transaction(signature):
    """Fetches transaction details asynchronously and extracts relevant transfers."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTransaction",
            "params": [signature, "jsonParsed"]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response and "meta" in response["result"]:
            process_transaction(response["result"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transaction details: {e}")

def process_transaction(transaction):
    """Analyzes the transaction to detect rug pull patterns and track wallets."""
    try:
        if "meta" in transaction and "postTokenBalances" in transaction["meta"]:
            instructions = transaction["transaction"]["message"]["instructions"]
            for instruction in instructions:
                if "parsed" in instruction and "info" in instruction["parsed"]:
                    info = instruction["parsed"]["info"]
                    sender = info.get("source")
                    receiver = info.get("destination")
                    amount = float(info.get("lamports", 0)) / 1e9  # Convert from lamports to SOL

                    if sender and receiver:
                        print(f"🔍 Transfer detected: {amount} SOL from {sender} to {receiver}")

                        if sender in tracked_wallets:
                            tracked_wallets[receiver] = tracked_wallets[sender] - 0.1  # Decrease by 0.1 SOL
                            print(f"📌 Tracking {receiver}, expected balance: {tracked_wallets[receiver]} SOL")

                            if tracked_wallets[receiver] == 3.7:
                                print(f"🚀 Token Creator Wallet Identified: {receiver}")
                                track_token_creation(receiver)
                            elif tracked_wallets[receiver] == 0.7:
                                print(f"💀 Rug Pull Wallet Identified: {receiver}")
                                track_rug_pull(receiver)

                        if amount in TRACKED_AMOUNTS:
                            tracked_wallets[receiver] = amount + 0.1  # Initialize tracking
                            print(f"👀 Tracking new suspicious transfer chain: {receiver}")
    except Exception as e:
        print(f"[ERROR] Failed to process transaction: {e}")

def track_token_creation(wallet):
    """Monitors the wallet to detect when it creates a new SPL token."""
    print(f"🔎 Monitoring {wallet} for SPL token creation...")
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getConfirmedSignaturesForAddress2",
        "params": [wallet, {"limit": 10}]
    }
    response = requests.post(RPC_URL, json=payload).json()
    if "result" in response:
        for tx in response["result"]:
            print(f"🆕 SPL Token Creation Detected: {tx['signature']}")

def track_rug_pull(wallet):
    """Monitors the rug pull wallet for large transfers."""
    print(f"🚨 Monitoring {wallet} for large fund movements...")
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getConfirmedSignaturesForAddress2",
        "params": [wallet, {"limit": 10}]
    }
    response = requests.post(RPC_URL, json=payload).json()
    if "result" in response:
        for tx in response["result"]:
            print(f"💰 Large fund movement detected: {tx['signature']}")

async def main():
    """Runs the WebSocket listener asynchronously."""
    await subscribe_to_account(TARGET_WALLET)

# Run the event loop in Colab safely
loop = asyncio.get_event_loop()
if loop.is_running():
    print("Event loop already running. Running in background...")
    task = loop.create_task(main())
else:
    loop.run_until_complete(main())


Event loop already running. Running in background...


In [ ]:
import asyncio
import json
import requests
import websockets
import nest_asyncio

# Apply nest_asyncio to avoid loop issues in Google Colab
nest_asyncio.apply()

# Public Solana RPC endpoint
RPC_URL = "https://api.mainnet-beta.solana.com"
WS_URL = "wss://api.mainnet-beta.solana.com"

# Target wallet to monitor
TARGET_WALLET = "HYoXu9V94BxGCWmiE47R5B6Zcb6seXPjQjhkKcKP5gBW"

# Define thresholds
TRACKED_AMOUNTS = {-4: 3.7, -1: 0.7}  # Starting and destination amounts
LIQUIDITY_THRESHOLD = -300  # Large liquidity deposits
LARGE_TRANSFER_THRESHOLD = 50  # Large movement detection

# Store wallet tracking information
tracked_wallets = {}

def fetch_transaction_details(wallet_address):
    """Fetches the latest transactions for the target wallet asynchronously."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getSignaturesForAddress",
            "params": [wallet_address, {"limit": 5}]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response:
            for tx in response["result"]:
                fetch_transaction(tx["signature"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transactions: {e}")

def fetch_transaction(signature):
    """Fetches transaction details asynchronously and extracts relevant transfers."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTransaction",
            "params": [signature, "jsonParsed"]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response and "meta" in response["result"]:
            process_transaction(response["result"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transaction details: {e}")

def process_transaction(transaction):
    """Analyzes the transaction to detect rug pull patterns and track wallets."""
    try:
        if "meta" in transaction and "postTokenBalances" in transaction["meta"]:
            instructions = transaction["transaction"]["message"]["instructions"]
            for instruction in instructions:
                if "parsed" in instruction and "info" in instruction["parsed"]:
                    info = instruction["parsed"]["info"]
                    sender = info.get("source")
                    receiver = info.get("destination")
                    amount = float(info.get("lamports", 0)) / 1e9  # Convert from lamports to SOL

                    if sender and receiver:
                        print(f"🔍 Transfer detected: {amount} SOL from {sender} to {receiver}")

                        if sender in tracked_wallets:
                            tracked_wallets[receiver] = tracked_wallets[sender] - 0.1  # Decrease by 0.1 SOL
                            print(f"📌 Tracking {receiver}, expected balance: {tracked_wallets[receiver]} SOL")

                            if tracked_wallets[receiver] == 3.7:
                                print(f"🚀 Token Creator Wallet Identified: {receiver}")
                                track_token_creation(receiver)
                            elif tracked_wallets[receiver] == 0.7:
                                print(f"💀 Rug Pull Wallet Identified: {receiver}")
                                track_rug_pull(receiver)

                        if amount in TRACKED_AMOUNTS:
                            tracked_wallets[receiver] = amount + 0.1  # Initialize tracking
                            print(f"👀 Tracking new suspicious transfer chain: {receiver}")
    except Exception as e:
        print(f"[ERROR] Failed to process transaction: {e}")

def track_token_creation(wallet):
    """Monitors the wallet to detect when it creates a new SPL token."""
    print(f"🔎 Monitoring {wallet} for SPL token creation...")
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getConfirmedSignaturesForAddress2",
        "params": [wallet, {"limit": 10}]
    }
    response = requests.post(RPC_URL, json=payload).json()
    if "result" in response:
        for tx in response["result"]:
            tx_details = fetch_transaction(tx["signature"])
            if "createAccount" in json.dumps(tx_details):
                print(f"🆕 SPL Token Creation Detected: {tx['signature']}")

def track_rug_pull(wallet):
    """Monitors the rug pull wallet for large transfers."""
    print(f"🚨 Monitoring {wallet} for large fund movements...")
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getConfirmedSignaturesForAddress2",
        "params": [wallet, {"limit": 10}]
    }
    response = requests.post(RPC_URL, json=payload).json()
    if "result" in response:
        for tx in response["result"]:
            tx_details = fetch_transaction(tx["signature"])
            if any(amount > LARGE_TRANSFER_THRESHOLD for amount in [float(inst["parsed"]["info"].get("lamports", 0)) / 1e9 for inst in tx_details["transaction"]["message"]["instructions"] if "parsed" in inst and "info" in inst["parsed"]]):
                print(f"💰 Large fund movement detected: {tx['signature']}")

async def main():
    """Runs the WebSocket listener asynchronously."""
    await subscribe_to_account(TARGET_WALLET)

# Run the event loop in Colab safely
loop = asyncio.get_event_loop()
if loop.is_running():
    print("Event loop already running. Running in background...")
    task = loop.create_task(main())
else:
    loop.run_until_complete(main())


Event loop already running. Running in background...


## Monitor Wallet Activity and detect token creation and liquidity

In [1]:
# IMPORTS AND SET-UP
import asyncio
import json
import requests
import websockets
import nest_asyncio

# Apply nest_asyncio to avoid loop issues in Google Colab
nest_asyncio.apply()

# Public Solana RPC endpoint
RPC_URL = "https://api.mainnet-beta.solana.com"
WS_URL = "wss://api.mainnet-beta.solana.com"

# Target wallet to monitor
TARGET_WALLET = "HYoXu9V94BxGCWmiE47R5B6Zcb6seXPjQjhkKcKP5gBW"

# Define thresholds
TRACKED_AMOUNTS = {-4: 3.7, -1: 0.7}  # Starting and destination amounts
LIQUIDITY_THRESHOLD = -300  # Large liquidity deposits
LARGE_TRANSFER_THRESHOLD = 50  # Large movement detection

# Store wallet tracking information
tracked_wallets = {}

In [2]:
# SUBSCRIBE TO ACCOUNT
async def subscribe_to_account(wallet_address):
    """Connects to Solana WebSocket and listens for transactions."""
    async with websockets.connect(WS_URL) as ws:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "accountSubscribe",
            "params": [wallet_address, {"encoding": "jsonParsed"}]
        }
        await ws.send(json.dumps(payload))

        print(f"Listening for transactions from {wallet_address}...")

        while True:
            try:
                response = await ws.recv()
                data = json.loads(response)

                if "result" in data and "value" in data["result"]:
                    await fetch_transaction_details(wallet_address)
            except Exception as e:
                print(f"[ERROR] WebSocket error: {e}")
                break  # Restart connection if needed

In [3]:
# GET TRANSACTION DETAILS
async def fetch_transaction_details(wallet_address):
    """Fetches the latest transactions for the target wallet asynchronously."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getSignaturesForAddress",
            "params": [wallet_address, {"limit": 5}]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response:
            for tx in response["result"]:
                await fetch_transaction(tx["signature"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transactions: {e}")

In [4]:
# FETCH TRANSACTION
async def fetch_transaction(signature):
    """Fetches transaction details asynchronously and extracts relevant transfers."""
    try:
        payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "getTransaction",
            "params": [signature, "jsonParsed"]
        }
        response = requests.post(RPC_URL, json=payload).json()

        if "result" in response and "meta" in response["result"]:
            process_transaction(response["result"])
    except Exception as e:
        print(f"[ERROR] Failed to fetch transaction details: {e}")

In [5]:
# PROCESS TRANSACTION
def process_transaction(transaction):
    """Analyzes the transaction to detect rug pull patterns and track wallets."""
    try:
        if "meta" in transaction and "postTokenBalances" in transaction["meta"]:
            instructions = transaction["transaction"]["message"]["instructions"]
            for instruction in instructions:
                if "parsed" in instruction and "info" in instruction["parsed"]:
                    info = instruction["parsed"]["info"]
                    sender = info.get("source")
                    receiver = info.get("destination")
                    amount = float(info.get("lamports", 0)) / 1e9  # Convert from lamports to SOL

                    if sender and receiver:
                        print(f"🔍 Transfer detected: {amount} SOL from {sender} to {receiver}")

                        if sender in tracked_wallets:
                            tracked_wallets[receiver] = tracked_wallets[sender] - 0.1  # Decrease by 0.1 SOL
                            print(f"📌 Tracking {receiver}, expected balance: {tracked_wallets[receiver]} SOL")

                            if tracked_wallets[receiver] == 3.7:
                                print(f"🚀 Token Creator Wallet Identified: {receiver}")
                                track_token_creation(receiver)
                            elif tracked_wallets[receiver] == 0.7:
                                print(f"💀 Rug Pull Wallet Identified: {receiver}")
                                track_rug_pull(receiver)

                        if amount in TRACKED_AMOUNTS:
                            tracked_wallets[receiver] = amount + 0.1  # Initialize tracking
                            print(f"👀 Tracking new suspicious transfer chain: {receiver}")
    except Exception as e:
        print(f"[ERROR] Failed to process transaction: {e}")

In [ ]:
# TRACK TOKEN CREATION & TRIGGER BUY FUNCTION
def track_token_creation(wallet):
    """Monitors the wallet to detect SPL token creation and triggers buy function."""
    print(f"🔎 Monitoring {wallet} for SPL token creation...")

    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getSignaturesForAddress",
        "params": [wallet, {"limit": 10}]
    }
    response = requests.post(RPC_URL, json=payload).json()

    if "result" in response:
        for tx in response["result"]:
            tx_details = fetch_transaction(tx["signature"])
            if not tx_details:
                continue

            instructions = tx_details.get("transaction", {}).get("message", {}).get("instructions", [])
            for instr in instructions:
                if instr.get("programId") == "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA":
                    if "initializeMint" in json.dumps(instr):
                        mint_address = instr.get("accounts", [None])[0]
                        if mint_address:
                            print(f"🆕 New SPL Token Created! Mint Address: {mint_address}")

                            # 🛒 Automatically trigger buy function
                            auto_buy_token(mint_address)
                            return mint_address
    return None  # No token found

In [ ]:
async def auto_buy_token(mint_address):
    """Automatically buys the newly created token using Raydium swap."""
    print(f"🛒 Attempting to buy token: {mint_address}")

    try:
        # Raydium Swap API - Example payload
        swap_payload = {
            "jsonrpc": "2.0",
            "id": 1,
            "method": "route",
            "params": {
                "inputMint": "So11111111111111111111111111111111111111112",  # SOL
                "outputMint": mint_address,  # Token to buy
                "inAmount": int(0.03 * 1e9),  # 0.03 SOL (adjustable)
                "slippage": 1.0,  # 1% slippage
                "userPublicKey": str(payer.pubkey()),
                "maxAccounts": 6
            }
        }

        # Send swap request
        response = requests.post("https://api.raydium.io/v2/swap", json=swap_payload)
        result = response.json()

        if "result" in result:
            print(f"✅ Swap executed successfully! TX: {result['result']['txid']}")
        else:
            print(f"⚠️ Swap failed: {result}")

    except Exception as e:
        print(f"[ERROR] Failed to execute swap: {e}")

In [7]:
# TRACK RUG PULL WALLET
def track_rug_pull(wallet):
    """Monitors the rug pull wallet for large transfers."""
    print(f"🚨 Monitoring {wallet} for large fund movements...")
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getConfirmedSignaturesForAddress2",
        "params": [wallet, {"limit": 10}]
    }
    response = requests.post(RPC_URL, json=payload).json()
    if "result" in response:
        for tx in response["result"]:
            tx_details = fetch_transaction(tx["signature"])
            if any(amount > LARGE_TRANSFER_THRESHOLD for amount in [float(inst["parsed"]["info"].get("lamports", 0)) / 1e9 for inst in tx_details["transaction"]["message"]["instructions"] if "parsed" in inst and "info" in inst["parsed"]]):
                print(f"💰 Large fund movement detected: {tx['signature']}")

In [8]:
async def main():
    """Runs the WebSocket listener asynchronously."""
    await subscribe_to_account(TARGET_WALLET)

# Run the event loop in Colab safely
loop = asyncio.get_event_loop()
if loop.is_running():
    print("Event loop already running. Running in background...")
    task = loop.create_task(main())
else:
    loop.run_until_complete(main())

Event loop already running. Running in background...


# Automate token purchase

In [ ]:
# ENV
#RPC_HTTPS_URL="https://mainnet.helius-rpc.com/?api-key=" #Your RPC URL
#PrivateKey=""  #YOur Private KEy
#WSOL_TokenAccount="4mncAGe5Re1dnX1K6mPsKPQDmJijLrHrQxT3C4fShtjU" #Your WSOL Token Account
#Amount_to_Wrap=0.03  # Amount of SOL --> WSOL

In [ ]:
# WRAP SOL (Note: modify code to run it on colab)
import asyncio
import sys
import base58
from solana.rpc.api import Client
# from solana.rpc.api import Keypair
from solders.keypair import Keypair

from solana.rpc.types import TokenAccountOpts, TxOpts
# from solders.transaction import Transaction
# from solders.transaction import Transaction as SoldersTransaction

# from solana.transaction import Transaction
from solders.transaction import Transaction

from solders.compute_budget import set_compute_unit_price, set_compute_unit_limit
from solders.system_program import transfer, TransferParams
from solders.pubkey import Pubkey
from spl.token.instructions import create_associated_token_account, SyncNativeParams
from spl.token.constants import WRAPPED_SOL_MINT, TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import sync_native
from solana.rpc.commitment import Commitment, Confirmed
from solana.rpc.async_api import AsyncClient

from solders.transaction import VersionedTransaction
from solders.message import MessageV0

from dotenv import dotenv_values
config = dotenv_values(".env")
solana_client = Client(config["RPC_HTTPS_URL"])
async_solana_client = AsyncClient(config["RPC_HTTPS_URL"])



# Initialize Solana client


private_key_string = config["PrivateKey"]
private_key_bytes = base58.b58decode(private_key_string)
payer = Keypair.from_bytes(private_key_bytes)
print(payer.pubkey())



mint_address = "So11111111111111111111111111111111111111112"

class style():
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    UNDERLINE = '\033[4m'
    RESET = '\033[0m'

def get_specific_token_account(owner_pubkey: str, mint_pubkey: str):
        owner_pubkey_obj = Pubkey.from_string(owner_pubkey)
        mint_pubkey_obj = Pubkey.from_string(mint_pubkey)
        opts = TokenAccountOpts(mint=mint_pubkey_obj)
        response =  solana_client.get_token_accounts_by_owner(owner_pubkey_obj, opts)
        if response.value is not None and len(response.value) > 0:
            return response.value[0].pubkey  # Return the first account found
        return None

wallet_solToken_acc= get_specific_token_account(str(payer.pubkey()),mint_address)

createWSOL_Acc = create_associated_token_account(payer.pubkey(),owner=payer.pubkey(),mint=WRAPPED_SOL_MINT)

wsol_token_account= createWSOL_Acc.accounts[1].pubkey

print(f" Your WSOL token Account: {wsol_token_account}")
# Amount of SOL to wrap (in lamports, 1 SOL = 1,000,000,000 lamports)
amount_to_wrap = int(float(config['Amount_to_Wrap']) * 10**9)
params_sync = SyncNativeParams(
    program_id=TOKEN_PROGRAM_ID,
    account=wsol_token_account
)


params = TransferParams(
    from_pubkey=payer.pubkey(),
    to_pubkey=wsol_token_account,
    lamports=amount_to_wrap
)


instructions=[]


if  wallet_solToken_acc is None:
    instructions.extend([
        createWSOL_Acc,
        transfer(params),
        sync_native(params_sync),
        set_compute_unit_price(100_750),
        set_compute_unit_limit(1_000_000)

    ])
else:
    instructions.extend([
        #createWSOL_Acc,
        transfer(params),
        sync_native(params_sync),
        set_compute_unit_price(498_750),
        set_compute_unit_limit(4_000_000)
    ])





async def send_and_confirm_transaction(client, payer, max_attempts=3):
        attempts = 0
        while attempts < max_attempts:
            try:

                compiled_message = MessageV0.try_compile(
                    payer.pubkey(),
                    instructions,
                    [],
                    client.get_latest_blockhash().value.blockhash,
                )
                print("Sending transaction...")
                txn = await async_solana_client.send_transaction(
                    txn=VersionedTransaction(compiled_message, [payer]),
                    opts=TxOpts(skip_preflight=True),
                )
                print("Transaction Signature:", txn.value)



                txid_string_sig = txn.value
                if txid_string_sig:
                    print("Transaction sent")
                    print(style.RED, f"Transaction Signature Waiting to be confirmed: https://solscan.io/tx/{txid_string_sig}" + style.RESET)
                    print("Waiting Confirmation")

                confirmation_resp = await async_solana_client.confirm_transaction(
                    txid_string_sig,
                    commitment=Confirmed,
                    sleep_seconds=0.5,
                )

                if confirmation_resp.value[0].err == None and str(
                        confirmation_resp.value[0].confirmation_status) == "TransactionConfirmationStatus.Confirmed":
                    print(style.GREEN + "Transaction Confirmed", style.RESET)
                    print(style.GREEN, f"Transaction Signature: https://solscan.io/tx/{txid_string_sig}", style.RESET)
                    return

                else:
                    print("Transaction not confirmed")
                    return False
            except asyncio.TimeoutError:
                attempts += 1
                print(f"Attempt {attempts}: Transaction not confirmed within 20 seconds. Attempting to resend.")
                print(f"Transaction signature: https://solscan.io/tx/{txid_string_sig}")
        if attempts == max_attempts:
            print("Maximum attempts reached. Transaction could not be confirmed.")


asyncio.run(send_and_confirm_transaction(solana_client, payer))

In [ ]:
# BUY TOKEN WITH WRAP SOL
import asyncio
import datetime
import time
from solders.message import MessageV0
from solders.transaction import  VersionedTransaction
from solana.rpc.types import TokenAccountOpts, TxOpts
from solders.pubkey import Pubkey
from solana.rpc.commitment import  Confirmed
from solana.rpc.api import RPCException
from solana.rpc.api import Client
from solders.keypair import Keypair
from solders.compute_budget import set_compute_unit_price,set_compute_unit_limit

from utils.create_close_account import   get_token_account, make_swap_instruction
from utils.birdeye import getSymbol
from solana.rpc.async_api import AsyncClient
from utils.pool_information import gen_pool, getpoolIdByMint
import os
from dotenv import load_dotenv


# Load.env file
load_dotenv()

# config = dotenv_values(".env")

RPC_HTTPS_URL= os.getenv("RPC_HTTPS_URL")
solana_client = Client(os.getenv("RPC_HTTPS_URL"))

async_solana_client = AsyncClient(os.getenv("RPC_HTTPS_URL"))
payer=Keypair.from_base58_string(os.getenv("PrivateKey"))
Wsol_TokenAccount=os.getenv('WSOL_TokenAccount')


AMM_PROGRAM_ID = Pubkey.from_string('675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8')
SERUM_PROGRAM_ID = Pubkey.from_string('srmqPvymJeFKQ4zGQed1GFppgkRHL9kaELCbyksJtPX')
LAMPORTS_PER_SOL = 1000000000
MAX_RETRIES = 2
RETRY_DELAY = 3

class style():
    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    UNDERLINE = '\033[4m'
    RESET = '\033[0m'


def getTimestamp():
    while True:
        timeStampData = datetime.datetime.now()
        currentTimeStamp = "[" + timeStampData.strftime("%H:%M:%S.%f")[:-3] + "]"
        return currentTimeStamp

async def get_specific_token_account(owner_pubkey: str, mint_pubkey: str):
    async with AsyncClient(RPC_HTTPS_URL) as client:
        owner_pubkey_obj = Pubkey.from_string(owner_pubkey)
        mint_pubkey_obj = Pubkey.from_string(mint_pubkey)
        # Using get_token_accounts_by_owner to fetch token accounts
        opts = TokenAccountOpts(mint=mint_pubkey_obj)
        response = await client.get_token_accounts_by_owner(owner_pubkey_obj, opts)
        if len(response.value) ==1 :
            return response.value[0].pubkey  # Return the first account found
    return None




async def buy(solana_client, TOKEN_TO_SWAP_BUY, payer, amount):


    retry_count = 0
    while retry_count < MAX_RETRIES:
        try:
            # Re-init transaction preparation
            token_symbol, SOl_Symbol = getSymbol(TOKEN_TO_SWAP_BUY)
            mint = Pubkey.from_string(TOKEN_TO_SWAP_BUY)
            # mint= TOKEN_TO_SWAP_BUY

            try:
                print("Fetching pool keys...")


                tokenPool_ID = await getpoolIdByMint(mint, AsyncClient(RPC_HTTPS_URL, commitment=Confirmed))
                print(tokenPool_ID)
                if tokenPool_ID:
                    print("AMMID FOUND")

                    fetch_pool_key = await gen_pool(str(tokenPool_ID), AsyncClient(RPC_HTTPS_URL, commitment=Confirmed))
                    pool_keys = fetch_pool_key
                else:
                    print("AMMID NOT FOUND SEARCHING WILL BE FETCHING WITH RAYDIUM SDK")


                    #pool_keys = fetch_pool_keys(str(mint))
            except Exception as e:
                print(e)


            amount_in = int(amount * LAMPORTS_PER_SOL)

            swap_associated_token_address, swap_token_account_Instructions = get_token_account(solana_client, payer.pubkey(), mint)
            swap_tx = []
            WSOL_token_account = Pubkey.from_string(Wsol_TokenAccount)
            instructions_swap = make_swap_instruction(amount_in, WSOL_token_account, swap_associated_token_address, pool_keys, mint, solana_client, payer)
            if swap_token_account_Instructions != None:

                swap_tx.append(swap_token_account_Instructions)

            swap_tx.extend([instructions_swap,
                            set_compute_unit_price(498_750),
                            set_compute_unit_limit(4_000_000)])

            #Do not edit Compute Unit Price and Compute Unit Limit if you have no idea what you are doing. Current test confirm txn in 1-5s


            # Execute Transaction
            print("Execute Transaction...")
            compiled_message = MessageV0.try_compile(
                payer.pubkey(),
                swap_tx,
                [],
                solana_client.get_latest_blockhash().value.blockhash,
            )
            print("Sending transaction...")
            txn = await async_solana_client.send_transaction(
                txn=VersionedTransaction(compiled_message, [payer]),
                opts=TxOpts(skip_preflight=True),
            )
            print("Transaction Signature:", txn.value)
            txid_string_sig = txn.value
            if txid_string_sig:
                print("Transaction sent")
                print(getTimestamp())
                print(style.RED,f"Transaction Signature Waiting to be confirmed: https://solscan.io/tx/{txid_string_sig}"+style.RESET)
                print("Waiting Confirmation")

            block_height = solana_client.get_block_height(Confirmed).value
            print(f"Block height: {block_height}")


            confirmation_resp = await async_solana_client.confirm_transaction(
                txid_string_sig,
                commitment=Confirmed,
                sleep_seconds=0.5,
                last_valid_block_height=block_height + 50


            )


            if  confirmation_resp.value[0].err== None and str(confirmation_resp.value[0].confirmation_status) =="TransactionConfirmationStatus.Confirmed":
                print(style.GREEN+"Transaction Confirmed",style.RESET)
                print(style.GREEN,f"Transaction Signature: https://solscan.io/tx/{txid_string_sig}",style.RESET)

                return True

            else:
                print("Transaction not confirmed")
                return False

        except asyncio.TimeoutError:
            print("Transaction confirmation timed out. Retrying...")
            retry_count += 1
            time.sleep(RETRY_DELAY)
        except RPCException as e:
            print(f"RPC Error: [{e.args[0].message}]... Retrying...")
            retry_count += 1
            time.sleep(RETRY_DELAY)
        except Exception as e:
            print(f"Unhandled exception: {e}. Retrying...")
            retry_count += 1
            time.sleep(RETRY_DELAY)

    print("Failed to confirm transaction after maximum retries.")
    return False

async def main():
    #7GCihgDB8fe6KNjn2MYtkzZcRjQy3t9GHdC8uHYmW2hr
    #RUpbmGF6p42AAeN1QvhFReZejQry1cLkE1PUYFVVpnL

    token_toBuy="3WdmE9BAHgVyB1JNswSUcj6RmkxnsvfJTd6RFnQ4pump"
    print(payer.pubkey())
    await buy(solana_client, token_toBuy, payer, 0.00065)

asyncio.run(main())

In [ ]:
# WRAP SOL
from solana.rpc.api import Client
from solana.transaction import Transaction
from solana.system_program import transfer, TransferParams
from solana.rpc.types import TxOpts
from solana.keypair import Keypair
from spl.token.instructions import create_associated_token_account, sync_native
from spl.token.constants import TOKEN_PROGRAM_ID, WRAPPED_SOL_MINT
from solana.publickey import PublicKey

# Solana RPC endpoint
RPC_URL = "https://api.mainnet-beta.solana.com"
client = Client(RPC_URL)

# Load your wallet
private_key = [your_private_key_array]  # Replace with your private key array
wallet = Keypair.from_secret_key(bytes(private_key))

# Function to wrap SOL
def wrap_sol(amount_sol: float):
    amount_lamports = int(amount_sol * 1_000_000_000)  # Convert SOL to lamports
    wsol_account = PublicKey.find_program_address([
        bytes(wallet.public_key), bytes(TOKEN_PROGRAM_ID)
    ], TOKEN_PROGRAM_ID)[0]

    transaction = Transaction()

    # Transfer SOL to WSOL account
    transaction.add(
        transfer(TransferParams(
            from_pubkey=wallet.public_key,
            to_pubkey=wsol_account,
            lamports=amount_lamports
        ))
    )

    # Sync WSOL balance
    transaction.add(
        sync_native(account=wsol_account)
    )

    # Send transaction
    response = client.send_transaction(transaction, wallet, opts=TxOpts(skip_preflight=True))
    print(f"Wrapped {amount_sol} SOL -> TX: {response}")
    return response

# Example usage
wrap_sol(1.0)  # Wrap 1 SOL

# Track volume and set auto-sell conditions